In [8]:
"""
security_id_map_builder.py

Fetches Dhan's instrument master CSV and builds a dict:
    { 'symbol_name': 'SEM_SMST_SECURITY_ID', ... }

Usage:
    python security_id_map_builder.py
"""

import requests
import pandas as pd
from io import StringIO

MASTER_URL = "https://images.dhan.co/api-data/api-scrip-master.csv"

def fetch_master_csv(url: str = MASTER_URL, timeout: int = 15) -> pd.DataFrame:
    """Download and return the instruments master as a pandas DataFrame."""
    resp = requests.get(url, timeout=timeout)
    resp.raise_for_status()
    df = pd.read_csv(StringIO(resp.text), low_memory=False, dtype=str)
    return df

def build_security_id_map(symbols_of_interest, col_symbol="SM_SYMBOL_NAME", col_id="SEM_SMST_SECURITY_ID"):
    """
    Build a dict mapping {symbol -> SEM_SMST_SECURITY_ID} for given symbols_of_interest.
    col_symbol can be "SM_SYMBOL_NAME" or "SEM_TRADING_SYMBOL".
    """
    df = fetch_master_csv()

    # Show first few rows so you know what names look like
    print("\nSample of available symbols:")
    print(df[[col_symbol, col_id]].head(20))

    if col_symbol not in df.columns:
        raise RuntimeError(f"Symbol column {col_symbol} not found in CSV.")
    if col_id not in df.columns:
        raise RuntimeError(f"ID column {col_id} not found in CSV.")

    sub = df[df[col_symbol].isin(symbols_of_interest)]
    mapping = sub.set_index(col_symbol)[col_id].to_dict()
    return mapping

if __name__ == "__main__":
    # Adjust this list after looking at the printed sample rows
    # For example, use "RELIANCE INDUSTRIES LTD" or "RELIANCE-EQ"
    my_symbols = [
        "RELIANCE INDUSTRIES LTD",
        "TATA CONSULTANCY SERVICES LTD"
    ]

    try:
        sec_map = build_security_id_map(my_symbols)
        print("\nsecurity_id_map = {")
        for sym, sid in sec_map.items():
            print(f'    "{sym}": "{sid}",')
        print("}")
    except Exception as e:
        print("Error:", e)



Sample of available symbols:
   SM_SYMBOL_NAME SEM_SMST_SECURITY_ID
0          USDINR              1026077
1          GBPINR              1026078
2          EURINR              1026079
3          JPYINR              1026080
4          EURUSD              1026081
5          GBPUSD              1026082
6          USDJPY              1026083
7          USDINR              1029516
8          GBPINR              1029517
9          EURINR              1029518
10         JPYINR              1029519
11         EURUSD              1029520
12         GBPUSD              1029521
13         USDJPY              1029522
14         USDINR              1034143
15         GBPINR              1034144
16         EURINR              1034145
17         JPYINR              1034146
18         EURUSD              1034147
19         GBPUSD              1034149

security_id_map = {
    "RELIANCE INDUSTRIES LTD": "2885",
}


In [9]:
import pandas as pd

df = pd.read_csv("https://images.dhan.co/api-data/api-scrip-master.csv", low_memory=False)
print(df[df["SM_SYMBOL_NAME"].str.contains("TATA", case=False, na=False)][["SM_SYMBOL_NAME","SEM_SMST_SECURITY_ID"]].head(20))


                   SM_SYMBOL_NAME  SEM_SMST_SECURITY_ID
46087          TATA POWER CO.LTD.                500400
46093             TATA ELXSI LTD.                500408
46119             TATA STEEL LTD.                500470
46123    TATA COMMUNICATIONS LTD.                500483
46135            TATA MOTORS LTD.                500570
46150         TATA CHEMICALS LTD.                500770
46154   TATA CONSUMER PRODUCTS LI                500800
46175   TATA INVESTMENT CORPORATI                501301
48242   TATA TELESERVICES (MAHARA                532371
48321   TATA CONSULTANCY SERVICES                532540
50524   Tata Technologies Limited                544028
51023   Tata Mutual Fund - Permit                590140
166317   TATA CONSULTANCY SERV LT                 11536
166612            TATAAML - NPBET                 12978
167540   TATA INVESTMENT CORP LTD                  1621
168631  TATA TECHNOLOGIES LIMITED                 20293
169033           TATAAML-TATAGOLD               

In [10]:
import pandas as pd

csv_url = "https://images.dhan.co/api-data/api-scrip-master.csv"
df = pd.read_csv(csv_url, low_memory=False)

# filter only cash equity
eq_df = df[
    (df["SEM_EXCH_INSTRUMENT_TYPE"].astype(str).str.upper() == "ES")
    & (df["SEM_INSTRUMENT_NAME"].astype(str).str.upper() == "EQUITY")
]

print("Total ES/EQUITY rows:", len(eq_df))

# if you want only NSE-listed codes (SEM_EXM_EXCH_ID == 'NSE')
nse_eq_df = eq_df[eq_df["SEM_EXM_EXCH_ID"].astype(str).str.upper() == "NSE"]
print("Total NSE ES/EQUITY rows:", len(nse_eq_df))


Total ES/EQUITY rows: 7775
Total NSE ES/EQUITY rows: 2683


In [12]:
import pandas as pd

csv_url = "https://images.dhan.co/api-data/api-scrip-master.csv"
df = pd.read_csv(csv_url, low_memory=False)

# sirf NSE + ES/EQUITY rows
nse_df = df[
    (df["SEM_EXM_EXCH_ID"].astype(str).str.upper() == "NSE")
    & (df["SEM_EXCH_INSTRUMENT_TYPE"].astype(str).str.upper() == "ES")
    & (df["SEM_INSTRUMENT_NAME"].astype(str).str.upper() == "EQUITY")
].copy()

# sirf relevant columns rakho
nse_df = nse_df[["SM_SYMBOL_NAME", "SEM_SMST_SECURITY_ID"]]

# save locally
nse_df.to_csv("nse_equities.csv", index=False, encoding="utf-8")

print("Saved NSE stock list to nse_equities.csv")


Saved NSE stock list to nse_equities.csv
